In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

# Haar cascades
- 머신러닝 기반의 object 검출 알고리즘
- 특징 벡터를 만들기 위해 하르 특징 사용
    - 하르 특징
        - 영상의 작은 패치들의 간단한 합과 차

# 통합 영상
Area of the rectangle **ABCD = AC - (AB+AD-AA)**

# Face Detection

In [8]:
face_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_frontalface_alt.xml')  # face detection xml
 
cap = cv2.VideoCapture('img/bruno.avi') 
scaling_factor = 0.5
size = (int(cap.get(3)),int(cap.get(4)))
# result = cv2.VideoWriter('avi_output/face_bruno.avi',cv2.VideoWriter_fourcc(*'MJPG'),10,size) # save code
while True: 
    ret, frame = cap.read() 
    frame = cv2.resize(frame, None, fx=1, 
 fy=1, interpolation=cv2.INTER_AREA)
 
    face_rects = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=3)  # 현 크기보다 1.3배 큰 사각형 그리기
    for (x,y,w,h) in face_rects: 
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3) 
        print((x,y,w,h))
 
    cv2.imshow('Face Detector', frame)
#     result.write(frame)
 
    c = cv2.waitKey(1) 
    if c == 27: 
        break 

cap.release()
# result.release()
cv2.destroyAllWindows() 

(191, 3, 212, 212)
(200, 9, 212, 212)
(198, 8, 212, 212)
(203, 17, 212, 212)
(201, 13, 212, 212)
(206, 26, 212, 212)
(209, 24, 212, 212)
(200, 25, 212, 212)
(185, 25, 239, 239)
(203, 41, 200, 200)
(180, 27, 230, 230)
(196, 39, 205, 205)
(154, 143, 109, 109)
(155, 141, 110, 110)
(158, 139, 111, 111)
(161, 139, 112, 112)
(165, 138, 109, 109)
(171, 139, 109, 109)
(172, 136, 111, 111)
(176, 136, 110, 110)
(176, 135, 112, 112)
(182, 133, 110, 110)
(183, 130, 112, 112)
(191, 128, 109, 109)
(192, 125, 110, 110)
(194, 124, 113, 113)
(199, 119, 109, 109)
(198, 115, 112, 112)
(196, 108, 113, 113)
(200, 105, 108, 108)
(197, 98, 114, 114)
(203, 100, 105, 105)
(202, 100, 107, 107)
(201, 97, 112, 112)
(202, 101, 108, 108)
(201, 98, 108, 108)
(201, 97, 108, 108)
(201, 64, 154, 154)
(217, 77, 138, 138)
(211, 77, 152, 152)
(209, 80, 156, 156)
(217, 90, 144, 144)
(214, 90, 147, 147)
(222, 98, 133, 133)
(226, 102, 133, 133)
(226, 99, 138, 138)
(222, 98, 144, 144)
(217, 96, 147, 147)
(208, 86, 163, 163)
(

# 얼굴에 마스크 씌우기

- cv2.threshold(src,thresh,maxval,type) -> thresh = 임계값, maxval = 임계값을 넘었을 때 적용할 value

In [3]:
face_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_frontalface_alt.xml')  # face detection xml
face_mask = cv2.imread('img/mask_hannibal.jpg')
h_mask,w_mask = face_mask.shape[:2]

if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')

cap = cv2.VideoCapture('img/bruno.avi')
scaling_factor = 0.5

while True:
    ret,frame = cap.read()
    frame = cv2.resize(frame,None,fx = scaling_factor,fy=scaling_factor,interpolation=cv2.INTER_AREA)
    face_rects = face_cascade.detectMultiScale(frame,scaleFactor = 1.3,minNeighbors=3) # detect to face
    for (x,y,w,h) in face_rects:
        if h<=0 or w<=0: pass
        # Adjust the height and weight parameters depending on the sizes and the locations. 
        
        h,w = int(1.4*h),int(1.0*w)
        y -= int(0.1*h)
        x = int(x)
        
        # Extract the region of interest from the image 
        frame_roi = frame[y:y+h, x:x+w] # roi 영역 추출
        face_mask_small = cv2.resize(face_mask,(w,h),interpolation=cv2.INTER_AREA)
        
        # Convert colot image to grayscale and threshold it
        gray_mask = cv2.cvtColor(face_mask_small,cv2.COLOR_BGR2GRAY) # convert gray
        ret,mask = cv2.threshold(gray_mask,180,255,cv2.THRESH_BINARY_INV) # 배경이 흰색이므로 180이상을 0으로 나머지는 255로
        
        # create an inverse mask
        mask_inv = cv2.bitwise_not(mask)
        
        try:
            # use the mask to extract the face mask region of interest
            masked_face = cv2.bitwise_and(face_mask_small, face_mask_small, mask=mask) 
            # Use the inverse mask to get the remaining part of the image 
            masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv) 
        except cv2.error as e:
#             print('Ignoring arithmentic exceptions: '+str(e))
            break
        
        frame[y:y+h,x:x+w] = cv2.add(masked_face,masked_frame)
    cv2.imshow('face detector',frame)
    
    c = cv2.waitKey(1)
    if c == 27:
        break
cap.release()
cv2.destroyAllWindows()

# Eye detection

#### faces = face_cascade.detectMultiScale(gray,scaleFactor = 1.3,minNeighbors = 1) 
-  minNeighbors = batch_size와 비슷한 의미
- 작게 잡을 수록 더 세밀하게 scan

In [4]:
face_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_frontalface_alt.xml')
eye_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_eye.xml')
if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')
if eye_cascade.empty():
    raise IOError('Unable to load the eye cascade classifier xml file')
cap = cv2.VideoCapture('img/bruno.avi')
ds_factor = 0.5
size = (int(cap.get(3)),int(cap.get(4)))
result = cv2.VideoWriter('avi_output/eye_bruno.avi',cv2.VideoWriter_fourcc(*'MJPG'),10,size) # save code
while True:
    ret,frame = cap.read()
    frame = cv2.resize(frame,None,fx=ds_factor,fy=ds_factor,interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor = 1.3,minNeighbors = 1) 
   
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h,x:x+w]
        roi_color = frame[y:y+h,x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (x_eye,y_eye,w_eye,h_eye) in eyes:
            center = (int(x_eye + 0.5*w_eye),int(y_eye+0.5*h_eye))
            radius = int(0.3*(w_eye + h_eye))
            color = (0,255,0)
            thickness = 3
            cv2.circle(roi_color,center,radius,color,thickness)
    cv2.imshow('Eye detector',frame)
    result.write(frame)
    c = cv2.waitKey(1)
    if c == 27:
        break
result.release()
cap.release()
cv2.destroyAllWindows()

# Ear Detection

In [9]:
left_ear_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_mcs_leftear.xml')
right_ear_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_mcs_rightear.xml')

if left_ear_cascade.empty():
    raise IOError('Unable to load the left ear cascade classifier xml file')
if right_ear_cascade.empty():
    raise IOError('Unable to load the right ear cascade classifier xml file')

cap = cv2.VideoCapture('img/bruno.avi')
scaling_factor = 0.5

while True:
    
    ret,frame = cap.read()
    frame = cv2.resize(frame,None,fx = scaling_factor,fy = scaling_factor,interpolation = cv2.INTER_AREA)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    left_ear = left_ear_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=3)
    right_ear = right_ear_cascade.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=3)
    
    for (x,y,w,h) in left_ear:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
    for (x,y,w,h) in right_ear:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
        
    cv2.imshow('Ear detector',frame)
    c= cv2.waitKey(1)
    if c== 27:
        break
cap.release()
cv2.destroyAllWindows()

- sample 영상에서 귀를 잘 못찾음

# Mouth Detection

In [13]:
mouth_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_mcs_mouth.xml')


if mouth_cascade.empty():
    raise IOError('Unable to load the left mouth cascade classifier xml file')

cap = cv2.VideoCapture('img/bruno.avi')
scaling_factor = 0.5

while True:
    
    ret,frame = cap.read()
    frame = cv2.resize(frame,None,fx = scaling_factor,fy = scaling_factor,interpolation = cv2.INTER_AREA)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    mouth_rects = mouth_cascade.detectMultiScale(gray, scaleFactor=1.7, minNeighbors=3)
    
    
    for (x,y,w,h) in mouth_rects:
#         y = int(y-0.15*h) # why..?
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
        break
        
    cv2.imshow('Mouth detector',frame)
    c= cv2.waitKey(1)
    if c== 27:
        break
cap.release()
cv2.destroyAllWindows()